In [2]:
import pandas as pd
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from transformers import BertTokenizerFast

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
df = pd.read_csv("/Users/pumpkin/Documents/Graduate_School/1st_Semester/ANLY_580/GU-ANLY580-PROJECT/cleaned_data/droppedDF.csv")
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df['selftext'] = df['selftext'].astype('str') 

In [5]:
datasetLab = df

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

batch_size = 1000

def batch_iterator():
    for i in range(0, len(datasetLab), batch_size):
        yield datasetLab[i : i + batch_size]["selftext"]

tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

tokenizer.decoder = decoders.WordPiece(prefix="##")

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

2 3


In [7]:
reddit_posts_bert = new_tokenizer(datasetLab['selftext'].tolist())

In [8]:
type(reddit_posts_bert)

transformers.tokenization_utils_base.BatchEncoding